# Classification of Leukemia Subtypes from Gene Expression Data

**Author:** Samir Kerkar  
**University of California, Irvine — B.S. Mathematics**

---

## Objective

This notebook demonstrates how machine learning can classify **acute myeloid leukemia (AML)** vs. **acute lymphoblastic leukemia (ALL)** from gene expression profiles measured via DNA microarray. Accurate subtype classification is critical — AML and ALL require fundamentally different treatment protocols, and misclassification can delay life-saving therapy.

We work with a classic high-dimensional dataset: **72 patients** × **7,129 genes**, creating a challenging *p >> n* problem where the feature space vastly exceeds the sample size. The notebook walks through:

1. Data preprocessing and exploration
2. Feature scaling and dimensionality reduction (PCA)
3. Training and comparison of 7 classification models
4. Evaluation with confusion matrices, ROC curves, and cross-validation
5. Feature importance analysis to identify discriminative genes

**Best result: F1 = 0.95 (SVM, Logistic Regression)**

---
## 1. Setup and Imports

In [ ]:
# Core
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Models
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

# Evaluation
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, f1_score, precision_recall_curve
)

# Plot settings
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.labelsize': 11,
    'axes.spines.top': False,
    'axes.spines.right': False,
})
COLORS = ['#2E86AB', '#A23B72', '#F18F01', '#2CA58D', '#E84855', '#6C5B7B', '#355C7D']
sns.set_palette(COLORS)

print('All libraries loaded.')

---
## 2. Data Loading

The dataset comes from the landmark Golub et al. (1999) study on molecular classification of cancer. It contains:
- **Training set:** 38 bone marrow samples (27 ALL, 11 AML)
- **Test set:** 34 samples (20 ALL, 14 AML)
- **Features:** 7,129 gene expression levels per sample

Each row in the raw data represents a gene, and each column represents a patient — so we need to transpose.

In [ ]:
# Load data — update paths to your local files or use URLs
gene_train = pd.read_csv('data_set_ALL_AML_train.csv')
gene_test = pd.read_csv('data_set_ALL_AML_independent.csv')
labels = pd.read_csv('actual.csv')

print(f'Training data shape: {gene_train.shape}')
print(f'Test data shape:     {gene_test.shape}')
print(f'Labels shape:        {labels.shape}')
print(f'\nClasses: {labels["cancer"].unique()}')
print(f'Class distribution:\n{labels["cancer"].value_counts()}')

---
## 3. Data Preprocessing

Several cleaning steps are needed:
1. Remove non-numeric columns (`Gene Description`, `Gene Accession Number`)
2. Remove `call` columns (presence/absence flags — not expression values)
3. Transpose so that rows = patients, columns = genes
4. Merge with labels

In [ ]:
# Remove non-expression columns
g_train = gene_train.loc[:, ~gene_train.columns.str.startswith('call')]
g_test = gene_test.loc[:, ~gene_test.columns.str.startswith('call')]

g_train = g_train.drop(['Gene Description', 'Gene Accession Number'], axis=1)
g_test = g_test.drop(['Gene Description', 'Gene Accession Number'], axis=1)

# Combine and transpose: rows become patients, columns become genes
all_data = pd.concat([g_train, g_test], axis=1).T
all_data['patient'] = all_data.index.astype(int)

# Encode target: ALL = 0, AML = 1
labels['Cancer'] = (labels['cancer'] == 'AML').astype(int)

# Merge features with labels
data = pd.merge(all_data, labels, on='patient')

print(f'Final dataset: {data.shape[0]} patients × {data.shape[1] - 3} genes')
print(f'Missing values: {data.isna().sum().sum()}')

---
## 4. Exploratory Data Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Class distribution
counts = data['cancer'].value_counts()
bars = axes[0].bar(counts.index, counts.values, color=[COLORS[0], COLORS[1]], 
                   edgecolor='white', linewidth=1.5)
for bar, val in zip(bars, counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                 str(val), ha='center', va='bottom', fontweight='bold', fontsize=12)
axes[0].set_title('Class Distribution', fontweight='bold')
axes[0].set_ylabel('Number of Patients')

# Gene expression distribution (sample of 100 genes)
sample_genes = data.iloc[:, :100]
axes[1].hist(sample_genes.values.flatten(), bins=60, color=COLORS[0], alpha=0.7, edgecolor='white')
axes[1].set_title('Gene Expression Distribution (100 genes)', fontweight='bold')
axes[1].set_xlabel('Expression Level')
axes[1].set_ylabel('Frequency')

# Expression range per gene (variance)
gene_vars = data.iloc[:, :7129].var().sort_values(ascending=False)
axes[2].plot(range(len(gene_vars)), gene_vars.values, color=COLORS[1], linewidth=0.8)
axes[2].fill_between(range(len(gene_vars)), gene_vars.values, alpha=0.2, color=COLORS[1])
axes[2].set_title('Gene Variance (Sorted)', fontweight='bold')
axes[2].set_xlabel('Gene Index (sorted by variance)')
axes[2].set_ylabel('Variance')
axes[2].set_yscale('log')

plt.tight_layout()
plt.show()

print(f'Total genes: {7129}')
print(f'Genes with variance > 1e4: {(gene_vars > 1e4).sum()}')
print(f'Genes with variance < 100: {(gene_vars < 100).sum()}')

---
## 5. Train/Test Split and Feature Scaling

With 7,129 features and only 72 samples, **standardization is essential** — features have wildly different scales, and distance-based algorithms (SVM, KNN) would be dominated by high-variance genes without it.

In [ ]:
# Separate features and target
X = data.drop(columns=['cancer', 'Cancer', 'patient'])
y = data['Cancer']

# Train/test split (stratified to maintain class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print(f'Training set: {X_train.shape[0]} samples')
print(f'Test set:     {X_test.shape[0]} samples')
print(f'Features:     {X_train.shape[1]}')
print(f'\nTrain class distribution: {dict(y_train.value_counts())}')
print(f'Test class distribution:  {dict(y_test.value_counts())}')

In [ ]:
# Standardize features (zero mean, unit variance)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Visualize effect of scaling
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

pd.DataFrame(X_train.iloc[:, :200]).plot(kind='kde', legend=False, ax=axes[0], 
                                          alpha=0.3, color=COLORS[0])
axes[0].set_title('Before Scaling (200 genes)', fontweight='bold')
axes[0].set_xlabel('Expression Value')
axes[0].set_xlim(-2000, 5000)

pd.DataFrame(X_train_scaled[:, :200]).plot(kind='kde', legend=False, ax=axes[1], 
                                            alpha=0.3, color=COLORS[1])
axes[1].set_title('After StandardScaler (200 genes)', fontweight='bold')
axes[1].set_xlabel('Standardized Value')
axes[1].set_xlim(-5, 5)

plt.tight_layout()
plt.show()

---
## 6. Dimensionality Reduction with PCA

With 7,129 features and only 54 training samples, we face the **curse of dimensionality**. PCA projects the data onto the directions of maximum variance, dramatically reducing the feature space while preserving the most informative signal.

In [ ]:
# Compute PCA on full feature set to analyze explained variance
pca_full = PCA()
pca_full.fit(X_train_scaled)

# Cumulative explained variance
cumulative_var = np.cumsum(pca_full.explained_variance_ratio_)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Individual + cumulative variance
n_show = min(40, len(pca_full.explained_variance_ratio_))
axes[0].bar(range(n_show), pca_full.explained_variance_ratio_[:n_show], 
            color=COLORS[0], alpha=0.7, label='Individual')
axes[0].plot(range(n_show), cumulative_var[:n_show], 'o-', 
             color=COLORS[1], linewidth=2, label='Cumulative')
axes[0].axhline(y=0.95, color='gray', linestyle='--', alpha=0.5, label='95% threshold')
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('PCA Explained Variance', fontweight='bold')
axes[0].legend()

# 3D PCA visualization
pca_3d = PCA(n_components=3)
X_3d = pca_3d.fit_transform(X_train_scaled)

ax3d = fig.add_subplot(122, projection='3d')
for label, color, name in [(0, COLORS[0], 'ALL'), (1, COLORS[1], 'AML')]:
    mask = y_train.values == label
    ax3d.scatter(X_3d[mask, 0], X_3d[mask, 1], X_3d[mask, 2], 
                 c=color, label=name, s=60, edgecolors='white', linewidth=0.5)
ax3d.set_xlabel('PC1')
ax3d.set_ylabel('PC2')
ax3d.set_zlabel('PC3')
ax3d.set_title('3D PCA Projection', fontweight='bold')
ax3d.legend()

plt.tight_layout()
plt.show()

# Find optimal number of components for 95% variance
n_components_95 = np.argmax(cumulative_var >= 0.95) + 1
print(f'Components for 95% variance: {n_components_95}')
print(f'PC1 explains: {pca_full.explained_variance_ratio_[0]:.1%}')
print(f'PC1-3 explain: {cumulative_var[2]:.1%}')

In [ ]:
# Apply PCA with optimal number of components
n_comp = min(n_components_95, X_train_scaled.shape[0] - 1)  # Can't exceed n_samples - 1
pca = PCA(n_components=n_comp)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f'Reduced from {X_train_scaled.shape[1]} features to {X_train_pca.shape[1]} principal components')
print(f'Variance retained: {sum(pca.explained_variance_ratio_):.1%}')

---
## 7. Model Training and Evaluation

We train 7 classification models and evaluate each with:
- **Accuracy** on the held-out test set
- **F1 score** (harmonic mean of precision and recall)
- **Confusion matrix** to visualize error patterns
- **5-fold stratified cross-validation** for robust performance estimates

In [ ]:
# Define models
models = {
    'SVM (Linear)': SVC(C=1, kernel='linear', probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5, weights='distance'),
    'Logistic Regression': LogisticRegression(C=0.01, solver='liblinear', max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(max_depth=3, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=50, max_depth=2, random_state=42),
    'Naive Bayes': GaussianNB(),
}

# Train and evaluate each model
results = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    # Fit and predict
    model.fit(X_train_pca, y_train)
    y_pred = model.predict(X_test_pca)
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cv_scores = cross_val_score(model, X_train_pca, y_train, cv=cv, scoring='f1')
    
    results.append({
        'Model': name,
        'Test Accuracy': acc,
        'Test F1': f1,
        'CV F1 (mean)': cv_scores.mean(),
        'CV F1 (std)': cv_scores.std(),
    })
    
    print(f'{name:25s} | Acc: {acc:.3f} | F1: {f1:.3f} | CV F1: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}')

results_df = pd.DataFrame(results).sort_values('Test F1', ascending=False).reset_index(drop=True)
results_df

### 7.1 Model Comparison Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart: Test accuracy and F1
x = np.arange(len(results_df))
width = 0.35
bars1 = axes[0].barh(x - width/2, results_df['Test Accuracy'], width, 
                      label='Accuracy', color=COLORS[0], alpha=0.85)
bars2 = axes[0].barh(x + width/2, results_df['Test F1'], width, 
                      label='F1 Score', color=COLORS[1], alpha=0.85)
axes[0].set_yticks(x)
axes[0].set_yticklabels(results_df['Model'])
axes[0].set_xlabel('Score')
axes[0].set_title('Test Set Performance', fontweight='bold')
axes[0].legend()
axes[0].set_xlim(0.5, 1.05)
axes[0].grid(axis='x', alpha=0.3)

# Cross-validation scores with error bars
axes[1].barh(results_df['Model'], results_df['CV F1 (mean)'], 
             xerr=results_df['CV F1 (std)'], color=COLORS[2], alpha=0.85,
             capsize=5, edgecolor='white', linewidth=1)
axes[1].set_xlabel('F1 Score')
axes[1].set_title('5-Fold Cross-Validation F1 (mean ± std)', fontweight='bold')
axes[1].set_xlim(0.5, 1.05)
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### 7.2 Confusion Matrices (Top 4 Models)

In [ ]:
top_models = ['SVM (Linear)', 'Logistic Regression', 'Random Forest', 'Gradient Boosting']

fig, axes = plt.subplots(1, 4, figsize=(18, 4))

for i, name in enumerate(top_models):
    model = models[name]
    y_pred = model.predict(X_test_pca)
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                xticklabels=['ALL', 'AML'], yticklabels=['ALL', 'AML'],
                cbar=False, linewidths=2, linecolor='white',
                annot_kws={'size': 14, 'fontweight': 'bold'})
    axes[i].set_title(name, fontweight='bold', fontsize=10)
    axes[i].set_xlabel('Predicted')
    if i == 0:
        axes[i].set_ylabel('Actual')

plt.suptitle('Confusion Matrices — Top 4 Models', fontweight='bold', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()

### 7.3 ROC Curves

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

for i, (name, model) in enumerate(models.items()):
    if hasattr(model, 'predict_proba'):
        y_proba = model.predict_proba(X_test_pca)[:, 1]
    else:
        y_proba = model.decision_function(X_test_pca)
    
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    
    ax.plot(fpr, tpr, color=COLORS[i % len(COLORS)], linewidth=2,
            label=f'{name} (AUC = {roc_auc:.3f})')

ax.plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.4, label='Random (AUC = 0.500)')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves — All Models', fontweight='bold')
ax.legend(loc='lower right', frameon=True, fancybox=True)
ax.grid(True, alpha=0.3)
ax.set_xlim([-0.02, 1.02])
ax.set_ylim([-0.02, 1.05])

plt.tight_layout()
plt.show()

---
## 8. Feature Importance Analysis

Which genes are most discriminative for distinguishing AML from ALL? We use the **SVM coefficients** (since our best model uses a linear kernel) to identify the genes that contribute most to the classification decision. We map the PCA-space coefficients back to the original gene space.

In [ ]:
# Map SVM coefficients back to original gene space through PCA
svm_model = models['SVM (Linear)']
coef_pca = svm_model.coef_[0]  # Coefficients in PCA space
coef_original = coef_pca @ pca.components_  # Map back to gene space

# Get gene importance (absolute coefficient magnitude)
gene_importance = pd.Series(np.abs(coef_original), index=X.columns)
gene_importance_signed = pd.Series(coef_original, index=X.columns)

# Top 20 most important genes
top_20 = gene_importance.nlargest(20)
top_20_signed = gene_importance_signed[top_20.index]

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Absolute importance
colors_abs = [COLORS[1] if v > 0 else COLORS[0] for v in top_20_signed.values]
axes[0].barh(range(20), top_20.values[::-1], color=colors_abs[::-1], 
             edgecolor='white', linewidth=0.5)
axes[0].set_yticks(range(20))
axes[0].set_yticklabels([f'Gene {g}' for g in top_20.index[::-1]], fontsize=8)
axes[0].set_xlabel('|Coefficient|')
axes[0].set_title('Top 20 Discriminative Genes (SVM)', fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Direction of discrimination
top_30_signed = gene_importance_signed.reindex(gene_importance.nlargest(30).index)
colors_dir = [COLORS[1] if v > 0 else COLORS[0] for v in top_30_signed.values]
axes[1].barh(range(30), top_30_signed.values[::-1], color=colors_dir[::-1],
             edgecolor='white', linewidth=0.5)
axes[1].set_yticks(range(30))
axes[1].set_yticklabels([f'Gene {g}' for g in top_30_signed.index[::-1]], fontsize=7)
axes[1].set_xlabel('Coefficient (+ = AML, - = ALL)')
axes[1].set_title('Top 30 Genes — Direction of Association', fontweight='bold')
axes[1].axvline(x=0, color='gray', linewidth=0.5)
axes[1].grid(axis='x', alpha=0.3)

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=COLORS[1], label='Higher → AML'),
                   Patch(facecolor=COLORS[0], label='Higher → ALL')]
axes[1].legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

print(f'Total genes analyzed: {len(gene_importance)}')
print(f'Genes with non-trivial contribution (|coef| > median): {(gene_importance > gene_importance.median()).sum()}')

---
## 9. Final Results Summary

In [ ]:
# Best model detailed report
best_model = models['SVM (Linear)']
y_pred_best = best_model.predict(X_test_pca)

print('=' * 60)
print('BEST MODEL: SVM (Linear Kernel)')
print('=' * 60)
print(f'\nTest Accuracy:  {accuracy_score(y_test, y_pred_best):.4f}')
print(f'Test F1 Score:  {f1_score(y_test, y_pred_best):.4f}')
print(f'\nClassification Report:')
print(classification_report(y_test, y_pred_best, target_names=['ALL', 'AML']))

---
## 10. Conclusion

### Key Findings

1. **SVM and Logistic Regression achieved F1 = 0.95** on leukemia subtype classification, demonstrating that the AML/ALL decision boundary is approximately linear in PCA-reduced gene expression space.

2. **PCA was essential for generalization.** The raw 7,129-dimensional feature space leads to severe overfitting with only 72 samples. Reducing to the top principal components (capturing 95% of variance) provided the regularization needed for robust classification.

3. **Regularized linear models matched or outperformed ensemble methods**, suggesting that the underlying biological signal separating AML from ALL is captured by a linear combination of gene expression features — consistent with the biological understanding that these are fundamentally distinct diseases.

4. **Feature importance analysis** identified specific genes that drive the classification, which could be validated against known AML/ALL biomarkers in future work.

### Limitations

- **Small sample size (n=72)** limits confidence in performance estimates. Cross-validation helps but cannot fully compensate.
- **PCA loses gene-level interpretability** — the principal components are linear combinations of all genes, making it harder to identify individual biomarkers directly.
- **Binary classification only** — a more granular model could distinguish AML/ALL subtypes (e.g., T-cell ALL vs. B-cell ALL).

### Clinical Relevance

These results support the feasibility of automated leukemia subtype classification from gene expression profiles. With larger datasets and more sophisticated feature selection (e.g., recursive feature elimination, LASSO), such models could serve as diagnostic support tools to accelerate treatment decisions.